In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from modules.csvLoader import csvLoader

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Définition de fonctions utiles

In [3]:
def drop_extreme(df, colonne, low_qt, high_qt):
    limite_basse, limite_haute = df[colonne].quantile([low_qt, high_qt])
    return df.query("{low}<=`{colonne}`<={high}".format(low=limite_basse,high=limite_haute,colonne=colonne))

## Lecture des csv

In [4]:
maison_filtre = csvLoader.load('maison')

Successfully loaded  maison.csv


## Observation sur les types de mutation et leur influence sur la valeur de la maison

In [5]:
mutations = maison_filtre['Nature mutation'].drop_duplicates().tolist()
for mutation in mutations:
    df = maison_filtre[maison_filtre['Nature mutation']==mutation]
    print('nb ', mutation[:15], ' :\t', len(df), '\tsoit\t', round(len(df)/len(maison_filtre)*100, 2), ' % des maisons' )

del(mutation, mutations, df)

print()
print('Valeur moyenne par type de bien')
maison_valeur = maison_filtre[['Nature mutation', 'Valeur fonciere']]
print( maison_valeur.groupby(['Nature mutation']).mean().head() )

del(maison_valeur)

print()
print(len(maison_filtre))
maison_filtre = maison_filtre[maison_filtre['Nature mutation'] == 'Vente']
print(len(maison_filtre))

nb  Vente  :	 489535 	soit	 99.55  % des maisons
nb  Vente en l'état  :	 253 	soit	 0.05  % des maisons
nb  Echange  :	 694 	soit	 0.14  % des maisons
nb  Adjudication  :	 1124 	soit	 0.23  % des maisons
nb  Vente terrain à  :	 132 	soit	 0.03  % des maisons
nb  Expropriation  :	 10 	soit	 0.0  % des maisons

Valeur moyenne par type de bien
                                    Valeur fonciere
Nature mutation                                    
Adjudication                           1.427672e+05
Echange                                1.791159e+05
Expropriation                          1.582530e+05
Vente                                  3.727974e+05
Vente en l'état futur d'achèvement     2.006223e+06

491748
489535


## Observation sur les ventes groupées/dupliquées

In [6]:
print(len(maison_filtre))
maison_filtre = drop_extreme(maison_filtre, 'Nombre pieces principales', 0.1, 0.9)
print(len(maison_filtre))

nb_pieces = maison_filtre['Nombre pieces principales'].drop_duplicates().tolist()
nb_pieces.sort()
for piece in nb_pieces:
    df = maison_filtre[maison_filtre['Nombre pieces principales']==piece]
    print(int(piece), ' pièces :', len(df), '\tsoit\t', round(len(df)/len(maison_filtre)*100, 2), ' % des maisons' )

del(nb_pieces, df, piece)

489535
411491
3  pièces : 92487 	soit	 22.48  % des maisons
4  pièces : 154406 	soit	 37.52  % des maisons
5  pièces : 115475 	soit	 28.06  % des maisons
6  pièces : 49123 	soit	 11.94  % des maisons


## Filtre sur les colonnes inutiles et concatenation des colonnes 'Surface Carrez' et 'Surface reelle'

In [7]:
# maison_filtre[maison_filtre['Surface'] < 140]['Surface'].hist(bins=50)

print(len(maison_filtre))
maison_filtre = drop_extreme(maison_filtre, 'Surface', 0.1, 0.9)
print(len(maison_filtre))
print(maison_filtre['Surface'].min(), maison_filtre['Surface'].max())

411491
372302


## Suppression des extremes de 'Surface terrain'

Note: aucune corrrelation notable à retirer, je drop la colonne après filtre

In [8]:
# maison_filtre[maison_filtre['Surface terrain'] < 960]['Surface terrain'].hist(bins=50)

print(len(maison_filtre))
maison_filtre = drop_extreme(maison_filtre, 'Surface terrain', 0.1, 0.90)
print(len(maison_filtre))
print(maison_filtre['Surface terrain'].min(), maison_filtre['Surface terrain'].max())

# maison_filtre = maison_filtre.drop(columns=['Surface terrain'])

372302
335083


## Suppression des extremes de 'Valeur fonciere'

In [10]:
# maison_filtre[maison_filtre['Valeur fonciere'] < 450000.0]['Valeur fonciere'].hist(bins=50)

print(len(maison_filtre))
maison_filtre = drop_extreme(maison_filtre, 'Valeur fonciere', 0.05, 0.95)
print(len(maison_filtre))
# print(maison_filtre['Valeur fonciere'].min(), maison_filtre['Valeur fonciere'].max())

174138
156783


## Observation du prix au m²

In [11]:
maison_filtre['Prix metre carre'] = maison_filtre['Valeur fonciere']/maison_filtre['Surface']
valeur_surface = maison_filtre.groupby('Code postal')['Prix metre carre'].mean()

#Generation du csv prix au metre carre
csvLoader.save(valeur_surface, 'prix_metre_maison.csv')

maison_filtre = maison_filtre.merge(valeur_surface, on='Code postal')
maison_filtre = maison_filtre.drop(columns=['Prix metre carre_x'])
maison_filtre = maison_filtre.rename(columns={'Prix metre carre_y':'Prix metre carre'})

# Je test d'éliminer les extrèmes de prix au m²
# print(len(maison_filtre))
# maison_filtre = drop_extreme(maison_filtre, 'Prix metre carre', 0.05, 0.95)
# print(len(maison_filtre))

maison_filtre = maison_filtre.drop(columns=['Code postal'])
# del(valeur_surface)

# print(maison_filtre['Prix metre carre'].min(), maison_filtre['Prix metre carre'].max())
# maison_filtre['Prix metre carre'].hist()

# Analyses et Graphs

## Relation de la surface du terrain (Y) avec le prix (X)

In [11]:
# nb_sample = round(0.1*len(maison_filtre))
# print('taille du sample : ', nb_sample)
# plot = maison_filtre.sample(n=nb_sample).plot.scatter(x='Surface terrain', y='Valeur fonciere', c='DarkBlue', s=1)
# del(nb_sample, plot)

## Relation de la surface (Y) avec le prix (X)

In [13]:
# nb_sample = round(0.1*len(maison_filtre))
# print('taille du sample : ', nb_sample)
# plot = maison_filtre.sample(n=nb_sample).plot.scatter(x='Surface', y='Valeur fonciere', c='DarkBlue', s=1)
# del(nb_sample, plot)
# maison_filtre['Surface'].hist(bins=10)

## Relation du prix au metre (Y) avec le prix (X)

In [14]:
# nb_sample = round(0.1*len(maison_filtre))
# print('taille du sample : ', nb_sample)
# plot = maison_filtre.sample(n=nb_sample).plot.scatter(x='Prix metre carre', y='Valeur fonciere', c='DarkBlue', s=1)
# del(nb_sample, plot)

In [15]:
# maison_filtre['Valeur fonciere'].hist(bins=15)

## Recherche de correlation

In [12]:
# matrice = maison_filtre.corr().round(2)
# sns.heatmap(data=matrice, annot=True)
# del(matrice)

In [13]:
print(len(maison_filtre))
csvLoader.save(maison_filtre, 'maison_filtre', save_index=False)

156783
